In [13]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [14]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias de Mariá"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas, 12 colunas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1

    # Verifica se a matriz de demanda está na forma padrão
    num_demand_lines = 0
    for i in range(demand_start_line, len(lines)):
        if lines[i].strip():  # Se a linha não estiver vazia, contamos como linha de demanda
            num_demand_lines += 1
        else:
            break  # Interrompe se encontrar uma linha vazia (ou um bloco separado)
    
    # Calcula o multiplicador, caso tenha mais linhas do que períodos
    if num_demand_lines == periods * 2:
        multiplier = 2  # A matriz de demanda tem o dobro de linhas
        # raise Exception("FUCKED") 
    else:
        multiplier = 1  # A matriz de demanda tem o número esperado de linhas (T)
    
    # Leitura inicial das demandas
    for i in range(periods * multiplier):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)
    
    # Agora, se multiplier == 2, precisamos combinar as linhas 1-12 com 13-24 corretamente
    if multiplier == 2:
        new_demand_matrix = []
        for i in range(periods):  # Para cada linha do primeiro bloco (1-12), combinar com (13-24)
            combined_demand = demand_matrix[i] + demand_matrix[i + periods]  # Junta a linha i com i+12
            new_demand_matrix.append(combined_demand)
        demand_matrix = new_demand_matrix  # Substitui a matriz de demandas pela combinada
    
    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Lendo os custos de transferência
    transfer_costs = []
    transfer_cost_line = demand_start_line + periods * multiplier
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()  # Verificar se a linha não está vazia antes de tentar ler
        if line:
            transfer_costs.append(float(line))
        transfer_cost_line += 1

    def create_transfer_cost_matrix(transfer_costs, num_plants):  # Criar a matriz de custos de transferência (simétrica)
        transfer_cost_matrix = np.zeros((num_plants, num_plants))  # Inicializar a matriz de zeros
        if len(transfer_costs) == 1:
            transfer_cost = transfer_costs[0]  # Se houver apenas um custo de transferência, aplicar para todos os pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_cost
                    transfer_cost_matrix[k, j] = transfer_cost
        else:
            idx = 0  # Se houver múltiplos custos, aplicar entre pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_costs[idx]
                    transfer_cost_matrix[k, j] = transfer_costs[idx]
                    idx += 1
        return transfer_cost_matrix

    transfer_costs = create_transfer_cost_matrix(transfer_costs, num_plants)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}

In [15]:
def local_branching(model, Z, incumbent_solution, max_iters=10, neighborhood_size=3, timelimit=300):
    """
    Implementa a heurística de Local Branching.

    Parâmetros:
    model: Modelo Gurobi inicial
    Z: Variáveis binárias de setup (i, j, t)
    incumbent_solution: Solução viável inicial
    max_iters: Máximo de iterações para a heurística
    neighborhood_size: Tamanho inicial da vizinhança
    timelimit: Limite de tempo para cada subproblema
    """
    # Definir o tempo limite para as iterações
    model.Params.timelimit = timelimit

    # Extrair a solução incumbente inicial (Z[i,j,t] incumbente)
    incumbent_Z = {(i, j, t): incumbent_solution[i, j, t] for i in range(len(I)) for j in range(len(J)) for t in range(len(T))}
    
    # Função para adicionar a restrição de Local Branching
    def add_local_branching_constraint(model, incumbent_Z, neighborhood_size):
        """
        Adiciona uma restrição de vizinhança local ao modelo em torno da solução incumbente Z.
        """
        expr = gp.LinExpr()
        for i in range(len(I)):
            for j in range(len(J)):
                for t in range(len(T)):
                    # Diferença entre a solução incumbente e a variável atual Z[i,j,t]
                    expr += (1 - Z[i, j, t]) if incumbent_Z[i, j, t] == 1 else Z[i, j, t]
        
        # Adiciona a restrição de vizinhança local
        model.addConstr(expr <= neighborhood_size, "local_branching")
    
    # Inicializar a solução incumbente e o valor objetivo
    best_obj_val = model.objVal
    best_solution = incumbent_solution

    # Iterar com Local Branching
    for iteration in range(max_iters):
        print(f"Iteração {iteration + 1}:")

        # Criar uma cópia do modelo original para adicionar as novas restrições
        local_model = model.copy()

        # Adicionar a restrição de vizinhança ao modelo local
        add_local_branching_constraint(local_model, incumbent_Z, neighborhood_size)

        # Resolver o modelo com a vizinhança restrita
        local_model.optimize()

        # Verificar se o modelo encontrou uma solução viável e melhor
        if local_model.status == GRB.OPTIMAL or local_model.status == GRB.TIME_LIMIT:
            new_obj_val = local_model.objVal
            if new_obj_val < best_obj_val:
                print(f"Nova solução melhor encontrada com valor objetivo: {new_obj_val}")
                best_obj_val = new_obj_val
                best_solution = {v.varName: v.x for v in local_model.getVars()}
                # Expandir a vizinhança (opcional, pode manter constante se preferir)
                neighborhood_size += 1
            else:
                print("Nenhuma melhoria nesta iteração. Diminuindo vizinhança.")
                # Reduzir a vizinhança se não houver melhorias
                neighborhood_size = max(1, neighborhood_size - 1)
        else:
            print("Nenhuma solução viável encontrada.")
            break

    return best_solution, best_obj_val

In [16]:
instancia = '../instancias/maria_desiree/AAA012650_4.dat'
data = read_dat_file(instancia)
# display(data)

In [17]:
# Modelo
m = gp.Model('Local branching')

# Conjuntos
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

# Parâmetros
# Demanda (i, j, t)
d = np.array(data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(data['capacities'])
# Tempo de produção (i, j)
b = np.array(data['production_time'])
# Tempo de setup (i, j)
f = np.array(data['setup_time'])
# Custo de produção (i, j)
c = np.array(data['production_cost'])
# Custo de setup (i, j)
s = np.array(data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(data['inventory_costs'])

# Variáveis de decisão
# Quantidade produzida (i, j, t)
X = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='X')
# Quantidade estocada (i, j, t)
Q = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='Q')
# # Quantidade transportada (i, j, k(um outro j), t)
W = m.addVars(I, J, J, T, vtype=GRB.CONTINUOUS, name='W')
# # Variável de setup (binária) (i, j, t)
Z = m.addVars(I, J, T, vtype=GRB.BINARY, name='Z')

# Função objetivo
expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] + 
                        sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I)
m.setObjective(expr_objetivo, sense=GRB.MINIMIZE)

# Restrições
# Balanço de estoque (revisar comportamento)
    # Período inicial
m.addConstrs((Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t == 0),
            name='restricao_balanco_estoque')
    # Demais períodos
m.addConstrs((Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t > 0),
            name='restricao_balanco_estoque')

# Restrição que obriga setup (validar o range do r)
m.addConstrs((X[i, j, t] <= min((cap[j, t] - f[i, j]) / b[i, j], sum(sum(d[i, k, r] for r in range(t, T[-1] + 1)) for k in J)) * Z[i, j, t] for i in I for j in J for t in T)
            , name='restricao_setup')

# Restrição de capacidade
m.addConstrs((sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t] for j in J for t in T)
         , name='restricao_capacidade')
;

''

In [18]:
m.Params.timelimit = 30  # Limite de tempo para a resolução inicial
m.optimize()

if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT:
    # Extrair a solução inicial incumbente
    incumbent_solution = {key: Z[key].x for key in Z.keys()}
    
    # Aplicar Local Branching para melhorar a solução
    best_solution, best_obj_val = local_branching(m, Z, incumbent_solution, max_iters=10, neighborhood_size=20, timelimit=60)

    # Exibir a melhor solução encontrada
    print(f"Melhor solução encontrada com valor objetivo: {best_obj_val}")
else:
    print("Modelo inicial não encontrou solução viável.")

Set parameter TimeLimit to value 30
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7272 rows, 32400 columns and 60900 nonzeros
Model fingerprint: 0x419ab527
Variable types: 28800 continuous, 3600 integer (3600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [2e-01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 0 rows and 3600 columns
Presolve time: 0.13s
Presolved: 7272 rows, 28800 columns, 60900 nonzeros
Variable types: 25200 continuous, 3600 integer (3600 binary)

Root relaxation: objective 6.655481e+05, 16640 iterations, 0.67 seconds (0.22 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

In [19]:
# Nome do teste
m.ModelName

'Local branching'

In [20]:
# Número de períodos da instância
len(T)

12

In [21]:
# Número de fábricas
len(J)

6

In [22]:
# Número de produtos
len(I)

50